# Recurrent Convolutional Neural Networks (R-CNN) for decoding neural signals acquired from an InterAxon Muse for

Dataset has already been preprocessed: [MindBigData.csv](https://drive.google.com/file/d/1S1ut3mR7poG22qZ25ngVzk8_dndgMn74/view)

Original: https://colab.research.google.com/drive/1q1mvIC1xgEuPNNvgASrAG7zie3qRjr-8

We are reimplementing their method in PyTorch to reproduce the results.

In [ ]:
import math, random, torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
torch.cuda.is_available()

### Import dataset
* First column indicates from which electrode, the data was obtained from (i.e., FP1, FP2, TP9, TP10).
* Second column indicates the digit seen by test subject (ground truth label).
* Third column onwards represent the readings obtained from the electrodes.

In [ ]:
full_data = pd.read_csv("MindBigData.csv", header = None) ### code did not include first row of data, but I included here
full_data.head()

In [ ]:
full_data[0].value_counts()

In [ ]:
full_data.shape

There are 408 features (for each EGG reading). Hmmm never distinguish FP1, FP2, TP9, TP10 for now...

### Model training & testing
WEIRD that originally here is how the data was obtained:
``` python
# where labels are {FP1, FP2, TP9, TP10}
# the data features are the EEG readings tgt with the digits seen by the test subject
data = full_data.iloc[:, 1:].values
labels = full_data.iloc[:, 0].values
```
I am unable to reproduce the result even when using their original code, model isn't learning at all.

Hence, I decided to vary the data used:
``` python
# where labels are {0,1,...,8,9}, I dropped -1
# the data features are the EEG readings (FP1, FP2, TP9, TP10 treated as input channels)
full_data = full_data[full_data[1]!=-1]
full_data_channels = np.array([full_data[full_data[0] == channel].iloc[:, 1:].values for channel in ["FP1", "FP2", "TP9", "TP10"]])
full_data_channels = full_data_channels.reshape((full_data_channels.shape[1], full_data_channels.shape[0], full_data_channels.shape[2]))
data = full_data_channels[:,:,1:]
labels = full_data_channels[:,0,0]
```

##### PyTorch implementation

In [ ]:
# Prepare dataset
full_data = full_data[full_data[1]!=-1]
full_data_channels = np.array([full_data[full_data[0] == channel].iloc[:, 1:].values for channel in ["FP1", "FP2", "TP9", "TP10"]])
full_data_channels = full_data_channels.reshape((full_data_channels.shape[1], full_data_channels.shape[0], full_data_channels.shape[2]))
data = full_data_channels[:,:,1:]
labels = full_data_channels[:,0,0]

print(f"Raw number of labels:\n{pd.Series(labels).value_counts()}")

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
one_hot_labels = to_categorical(encoded_labels)

# Split the data into training, testing, and validation sets
# (nv followed his one, only used a few for training for testing)
X_train, X_temp, y_train, y_temp = train_test_split(data, one_hot_labels, test_size=0.8, random_state=42, stratify = one_hot_labels)
print(f"\nSize of X_train:\n{X_train.shape}")
print(f"\nSize of y_train:\n{y_train.shape}")
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify = y_temp)

class MyDataset(Dataset):
    """Custom dataset."""

    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out = {'data': self.data[idx], 'label': self.label[idx]}
        return out

train_loader = DataLoader(MyDataset(X_train, y_train), batch_size=128, shuffle=True) ###

# Create model
class RCNN(nn.Module):
    def __init__(self):
        super(RCNN,self).__init__()
        # 4 x 408
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 4, out_channels = 64, kernel_size = 3, padding = 0, stride = 1),
            # 64 x 406
            nn.ReLU(),
            nn.MaxPool1d(kernel_size = 2)
            # 64 x 203
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 0, stride = 1),
            # 128 x 201
            nn.ReLU(),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 6, padding = 0, stride = 1),
            # 128 x 196
            nn.ReLU(),
            nn.MaxPool1d(kernel_size = 2)
            # 128 x 98
        )
        self.lstm = nn.LSTM(input_size = 128*98, hidden_size = 80) ### 128*98
        self.dense = nn.Sequential(
            nn.Linear(80,80), ###
            nn.ReLU(),
            nn.Linear(80,10) ###
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, x.shape[-1]*x.shape[-2]) # flatten
        x = self.lstm(x)[0]
        x = self.dense(x)
        # x = torch.nn.functional.log_softmax(x, dim=1)
        return x

In [ ]:
# check trainable parameters
for p in RCNN().parameters():
    print(p.shape)

In [ ]:
# test output shape
network = RCNN()
# convert to tensor
x = torch.from_numpy(X_train[0:1])
x = x.type(torch.float)
print(x.shape)
preds = network(x)
print(preds.shape)
print(preds)

In [ ]:
# training
def one_hot_ce_loss(outputs, targets):
    criterion = nn.CrossEntropyLoss()
    _, labels = torch.max(targets, dim=1)
    return criterion(outputs, labels)

network = RCNN()
if torch.cuda.is_available():
    network = network.cuda(1)

optimizer = optim.Adam(network.parameters(), lr=0.00001)

for epoch in range(25):
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        data, labels = batch["data"], batch["label"]
        data = data.type(torch.float)
        if torch.cuda.is_available():
            data = data.cuda(1)
            labels = labels.cuda(1)

        preds = network(data)
        loss = one_hot_ce_loss(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _,prelabels=torch.max(preds,dim=1)
        total_correct += (torch.max(labels, dim=1)[1] == torch.max(preds, dim=1)[1]).sum().item()
    accuracy = total_correct/len(X_train)
    print("Epoch:%d  ,  Loss:%f  , Train Accuracy:%f "%(epoch, total_loss, accuracy * 100))

In [ ]:
# see last batch of predictions and labels
torch.max(labels, dim=1)[1], torch.max(preds, dim=1)[1]